<a href="https://colab.research.google.com/github/wirapong/ESG_Thailand_Financial_Performance/blob/main/ESG_Thailand_Financial_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ESG → Financial Performance — Replication Template
# Paper: "The Impact of ESG Performance on Financial Performance: Evidence from Listed Companies in Thailand"
# Requirements: pandas, numpy, statsmodels, scipy, tabulate (optional)
# pip install pandas numpy statsmodels scipy tabulate
import pandas as pd
import numpy as np
import statsmodels.api as sm


In [ ]:
from pathlib import Path
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from scipy import stats

In [ ]:

# -----------------------------------------------------------------------------
# 0) INPUTS
# -----------------------------------------------------------------------------
# Expected CSV schema (wide/panel, one row per firm-year):
# firm_id, year, EBIT, CapitalEmployed, NetIncome, TotalAssets, ES, SS, GS, ESG, SIZE, LEVERAGE, ROCE, ROA
# If ROCE/ROA are missing, they will be computed from EBIT/CapitalEmployed and NetIncome/TotalAssets.

CSV_PATH = "Thailand_ESG__data_30102025.csv"  # <-- change to your file path
FIRM_COL = "firm_id"
TIME_COL = "year"

DEPENDENT_VARS = ["ROCE", "ROA"]
#ESG_INDEP_VARS = ["ES", "SS", "GS", "ESG"]  # article uses both pillar scores and total ESG
ESG_INDEP_VARS = ["ESG"]  # article uses both pillar scores and total ESG
CONTROL_VARS = ["SIZE", "LEVERAGE"]

In [ ]:
# -----------------------------------------------------------------------------
# 1) LOAD & PREP
# -----------------------------------------------------------------------------
df = pd.read_csv(CSV_PATH)

# Ensure numeric
for col in ["EBIT", "CapitalEmployed", "NetIncome", "TotalAssets"] + ESG_INDEP_VARS + CONTROL_VARS + ["ROCE", "ROA"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Compute ROCE, ROA if missing (article definitions)
if "ROCE" not in df.columns and {"EBIT", "CapitalEmployed"}.issubset(df.columns):
    df["ROCE"] = df["EBIT"] / df["CapitalEmployed"]

if "ROA" not in df.columns and {"NetIncome", "TotalAssets"}.issubset(df.columns):
    df["ROA"] = df["NetIncome"] / df["TotalAssets"]

# Drop rows with critical missing values
needed = list(set(DEPENDENT_VARS + ESG_INDEP_VARS + CONTROL_VARS))
df = df.dropna(subset=needed)

# Optional: clip/guard against division issues/outliers
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=needed)

In [ ]:
# -----------------------------------------------------------------------------
# 2) DESCRIPTIVE STATISTICS (Table 3 analogue)
# -----------------------------------------------------------------------------
def descriptive_table(frame, cols):
    out = []
    for c in cols:
        s = frame[c].dropna()
        out.append({
            "Variable": c,
            "Range": s.max() - s.min(),
            "Min": s.min(),
            "Max": s.max(),
            "Mean": s.mean(),
            "Std": s.std(ddof=1)
        })
    return pd.DataFrame(out)

#desc_cols = ["ROCE", "ROA", "ES", "SS", "GS", "ESG", "SIZE", "LEVERAGE"]
desc_cols = ["ROCE", "ROA", "ESG", "SIZE", "LEVERAGE"]
desc = descriptive_table(df, [c for c in desc_cols if c in df.columns])
print("\n=== Descriptive Statistics (like Table 3) ===")
print(desc.to_string(index=False))



In [ ]:
# -----------------------------------------------------------------------------
# 3) CORRELATION MATRIX & MULTICOLLINEARITY (VIF) (Table 5 analogue)
# -----------------------------------------------------------------------------
#corr_vars = ["ROCE", "ROA", "ES", "SS", "GS", "ESG", "SIZE", "LEVERAGE"]
corr_vars = ["ROCE", "ROA", "ESG", "SIZE", "LEVERAGE"]
corr_vars = [c for c in corr_vars if c in df.columns]
corr = df[corr_vars].corr()
print("\n=== Correlation Matrix (like Table 5) ===")
print(corr.round(3).to_string())

# VIF on predictors only (as in article, check threshold ~10)
X_for_vif = df[ESG_INDEP_VARS + CONTROL_VARS].dropna()
X_for_vif_const = sm.add_constant(X_for_vif)
vif_list = []
for i, col in enumerate(X_for_vif_const.columns):
    if col == "const":  # skip intercept
        continue
    vif_val = variance_inflation_factor(X_for_vif_const.values, i)
    tol = 1.0 / vif_val if vif_val != 0 else np.nan
    vif_list.append({"Variable": col, "VIF": vif_val, "Tolerance": tol})
vif_df = pd.DataFrame(vif_list)
print("\n=== Multicollinearity Test (VIF & Tolerance) ===")
print(vif_df.round(3).to_string(index=False))

In [ ]:
# -----------------------------------------------------------------------------
# 4) UNIT ROOT (ADF) TESTS (Table 4 analogue)
# Note: The paper applies ADF to make series stationary. With short panels (e.g., T=3),
# ADF is not very informative; still, we show pooled-series ADF as an approximation.
# For better panel stationarity testing, consider panel unit root tests (e.g., IPS/LLC).
# -----------------------------------------------------------------------------
def pooled_series(frame, col, firm_col=FIRM_COL, time_col=TIME_COL):
    # Stack all firm time series end-to-end in time order for a rough pooled ADF
    tmp = frame[[firm_col, time_col, col]].dropna().sort_values([firm_col, time_col])
    return tmp[col].astype(float).values

print("\n=== Augmented Dickey–Fuller (ADF) — pooled approximation ===")
adf_rows = []
#for col in ["ES","SS","GS","ESG","SIZE","LEVERAGE","ROCE","ROA"]:
for col in ["ESG","SIZE","LEVERAGE","ROCE","ROA"]:
    if col in df.columns:
        series = pooled_series(df, col)
        if len(series) > 10:  # ADF needs reasonable length
            adf_stat, pval, _, _, crit, _ = adfuller(series, autolag="AIC")
            adf_rows.append({
                "Variable": col,
                "ADF t-stat": adf_stat,
                "p-value": pval
            })
        else:
            adf_rows.append({"Variable": col, "ADF t-stat": np.nan, "p-value": np.nan})
adf_df = pd.DataFrame(adf_rows)
print(adf_df.round(4).to_string(index=False))

In [ ]:
# -----------------------------------------------------------------------------
# 5) GRANGER CAUSALITY TESTS (Table 6 analogue)
# The article tests whether ESG components "Granger-cause" financial metrics.
# We'll run tests for (ES, SS, GS, ESG, SIZE, LEVERAGE) → (ROCE, ROA)
# by pooling series (approximation). Choose maxlag=2 as a reasonable default.
# -----------------------------------------------------------------------------
def granger_pooled_causality(frame, cause, effect, maxlag=2):
    """
    Rough pooled Granger: concatenate firm series; run on 2-column DataFrame [effect, cause]
    Returns min p-value across lags for F-test of Granger causality.
    """
    cols = [cause, effect, FIRM_COL, TIME_COL]
    gdf = frame[cols].dropna().sort_values([FIRM_COL, TIME_COL]).reset_index(drop=True)
    # build 2-col data in correct order for statsmodels: [effect, cause]
    data = gdf[[effect, cause]].astype(float)
    try:
        res = grangercausalitytests(data, maxlag=maxlag, verbose=False)
        # collect p-values for ssr_ftest across lags
        pvals = [res[lag][0]["ssr_ftest"][1] for lag in range(1, maxlag+1)]
        return float(np.nanmin(pvals)), pvals
    except Exception as e:
        return np.nan, []

print("\n=== Granger Causality (pooled approximation; p-values) ===")
#causes = [c for c in ["ES","SS","GS","ESG","SIZE","LEVERAGE"] if c in df.columns]
causes = [c for c in ["ESG","SIZE","LEVERAGE"] if c in df.columns]
effects = [c for c in ["ROCE","ROA"] if c in df.columns]
granger_results = []
for eff in effects:
    for cau in causes:
        pmin, all_p = granger_pooled_causality(df, cause=cau, effect=eff, maxlag=2)
        granger_results.append({"Cause → Effect": f"{cau} → {eff}", "min p-val (lags≤2)": pmin, "all p-vals": all_p})
granger_df = pd.DataFrame(granger_results)
print(granger_df.to_string(index=False))

In [ ]:
# -----------------------------------------------------------------------------
# 6) MULTIPLE REGRESSION — POOLED OLS (Models 1 & 2)
# Model 1: ROCE ~ ES + SS + GS + ESG + SIZE + LEVERAGE
# Model 2: ROA  ~ ES + SS + GS + ESG + SIZE + LEVERAGE
# -----------------------------------------------------------------------------
def pooled_ols(frame, y, Xcols):
    X = frame[Xcols].copy()
    yv = frame[y].astype(float)
    X = sm.add_constant(X.astype(float))
    model = sm.OLS(yv, X)
    fit = model.fit()  # you may use .fit(cov_type="HC3") for robust SEs
    # Durbin–Watson
    dw = sm.stats.stattools.durbin_watson(fit.resid)
    return fit, dw

def print_regression(fit, dw):
    print(f"\nR-squared: {fit.rsquared:.6f} | Adj R-squared: {fit.rsquared_adj:.6f}")
    print(f"F-statistic: {fit.fvalue:.6f} | Prob(F): {fit.f_pvalue:.6g}")
    print(f"Durbin–Watson: {dw:.6f}")
    print("\nCoefficients:")
    coefs = pd.DataFrame({
        "coef": fit.params,
        "std_err": fit.bse,
        "t": fit.tvalues,
        "p>|t|": fit.pvalues
    })
    print(coefs.round(6).to_string())

model_vars = [c for c in (ESG_INDEP_VARS + CONTROL_VARS) if c in df.columns]

if "ROCE" in df.columns:
    #print("\n=== Model 1: ROCE ~ ES + SS + GS + ESG + SIZE + LEVERAGE ===")
    print("\n=== Model 1: ROCE ~ ESG + SIZE + LEVERAGE ===")
    fit1, dw1 = pooled_ols(df, "ROCE", model_vars)
    print_regression(fit1, dw1)

if "ROA" in df.columns:
    #print("\n=== Model 2: ROA  ~ ES + SS + GS + ESG + SIZE + LEVERAGE ===")
    print("\n=== Model 2: ROA  ~ ESG + SIZE + LEVERAGE ===")
    fit2, dw2 = pooled_ols(df, "ROA", model_vars)
    print_regression(fit2, dw2)


In [ ]:
# -----------------------------------------------------------------------------
# 7) OPTIONAL — COEFFICIENT-FORM EQUATIONS (to mirror the paper’s presentation)
# -----------------------------------------------------------------------------
def equation_text(fit, yname):
    terms = [f"{coef:+.6f}*{name}" for name, coef in fit.params.items() if name != "const"]
    return f"{yname} = {fit.params['const']:+.6f} " + " ".join(terms)

if "ROCE" in df.columns:
    print("\nEquation (Model 1):")
    print(equation_text(fit1, "ROCE"))

if "ROA" in df.columns:
    print("\nEquation (Model 2):")
    print(equation_text(fit2, "ROA"))

In [ ]:
# -----------------------------------------------------------------------------
# 8) NOTES FOR REPLICATION FIDELITY
# -----------------------------------------------------------------------------
# • This script treats the data as pooled OLS, matching the article’s multiple regression setup.
# • The ADF and Granger sections implement pooled/concatenated approximations; with T=3 per firm,
#   time-series power is limited. For high-fidelity time-series inference, use longer panels or
#   panel-time-series methods (e.g., panel unit root / panel Granger).
# • Ensure ES, SS, GS, and ESG match your source methodology (e.g., CRISIL weights: E=35%, S=25%, G=40%).
# • SIZE (Total Assets) and LEVERAGE should follow the same definitions used when compiling your dataset.